In [345]:
import pandas as pd
import numpy as np 
import pathlib as Path
import os 
import glob
##combines all the player stat mvps from 1960 - 2010 (training data)
all_files = glob.glob("traincsvs/*.csv")
df_combinedcsv = pd.concat([pd.read_csv(f) for f in all_files], ignore_index = True)
df_withdrop = df_combinedcsv.drop_duplicates(keep='first', inplace=False, ignore_index=False)

In [346]:
##combines mvp list into one
df_trainmvp = pd.concat(map(pd.read_csv, ['mvp.csv', 'mvp2.csv']), ignore_index= True)


In [347]:
##merges the mvp list with the statistics for the training data from 1960-2010
df_finaltrain = pd.merge(df_withdrop, df_trainmvp, on='Year')

In [348]:
##test data from 2010-2019 (includes mvp)
df_testmvp = pd.read_csv('testmvp.csv')

In [349]:
##combines test statistics from 2010-2019 
all_testfiles = glob.glob("testcsvs/*.csv")
df_testcombinedcsv = pd.concat([pd.read_csv(f) for f in all_testfiles], ignore_index = True)
df_testwithdrop = df_testcombinedcsv.drop_duplicates(keep='first', inplace=False, ignore_index=False)

##merging the test data 
df_finaltest = pd.merge(df_testwithdrop, df_testmvp, on='Year')


In [350]:
##removing irrelevant columns in the dataset that do not affect if a player wins MVP - cleaning train data
df_temptrain = df_finaltrain.drop(['Rk', 'Pos', 'Age', 'Tm', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA', '2P%', 'FT%',  ], axis=1)
df_calculationstrain = df_temptrain.drop(['GS', 'DRB', 'ORB', 'PF'], axis=1)
df_temptest = df_finaltest.drop(['Rk', 'Pos', 'Age', 'Tm', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'FT', 'FTA', '2P%', 'FT%',], axis=1)
df_calculationstest = df_temptest.drop(['GS', 'DRB', 'ORB', 'PF'], axis=1)

In [351]:
##removed all rows with letters instead of numbers 
df_calculationstrain = df_calculationstrain[df_calculationstrain.Player!='Player']
df_calculationstest = df_calculationstest[df_calculationstest.Player!='Player']

##replace all null values with 0 
##null values exist because in the early years, some stats such as steals were not recorder by the NBA 
df_calculationstest = df_calculationstest.fillna(0)
df_calculationstrain = df_calculationstrain.fillna(0)

## remove * signs from players 
for i in range(len(df_calculationstrain.index)):
    if (df_calculationstrain.iloc[i, 1]) [len(df_calculationstrain.iloc[i, 1]) - 1] == "*":
        df_calculationstrain.iloc[i, 1] = (df_calculationstrain.iloc[i, 1]) [:-1]
    
for i in range(len(df_calculationstest.index)):
    if (df_calculationstest.iloc[i, 1]) [len(df_calculationstest.iloc[i, 1]) - 1] == "*":
        df_calculationstest.iloc[i, 1] = (df_calculationstest.iloc[i, 1]) [:-1]
   



In [352]:
##change the mvp coloumn from name to 0 or 1 to indicate a boolean MVP value (yes, he is the mvp if 1)
df_calculationstrain['MVP'] = (df_calculationstrain['MVP'] == df_calculationstrain['Player']).astype(int)
df_calculationstest['MVP'] = (df_calculationstest['MVP'] == df_calculationstest['Player']).astype(int)

##useful to make another instance of the dataframe 
df_manual = df_calculationstest['Player']

##dropping the player column and turning the data into one type 
df_calculationstrain = df_calculationstrain.drop('Player', axis=1)
df_calculationstest = df_calculationstest.drop('Player', axis=1)

df_calculationstrain = df_calculationstrain.astype(float)
df_calculationstest = df_calculationstest.astype(float)

df_calculationstrain = df_calculationstrain[df_calculationstrain.PTS > 10]
df_calculationstest = df_calculationstest[df_calculationstest.PTS > 10]

df_calculationstrain = df_calculationstrain[df_calculationstrain.G > 40]
df_calculationstest = df_calculationstest[df_calculationstest.G > 40] 

df_modeltrain  = df_calculationstrain[df_calculationstrain.MP > 30]
df_modeltest = df_calculationstest[df_calculationstest.MP > 30] 

##new df to keep track of player name 
df_tracktest = pd.concat([df_calculationstrain, df_manual], axis=1, join='inner')
df_tracktest = df_tracktest.reset_index(drop=True)



In [353]:
##checks to see where the year changes (since the amount of players per year differs)

my_column_changestrain = df_modeltrain['Year'].shift() != df_modeltrain['Year']
my_column_changestest = df_modeltest['Year'].shift() != df_modeltest['Year']

my_column_changestrain = list(my_column_changestrain)
my_column_changestest = list(my_column_changestest)

listofyeartrain = []
listofyeartest = []

##creates a list which holds the indexes of when the year changes 
for i in range(len(my_column_changestrain)):
    if my_column_changestrain[i]:
        listofyeartrain.append(i)

for i in range(len(my_column_changestest)):
    if my_column_changestest[i]:
        listofyeartest.append(i)


##further cleaning the data 
df_modeltrain = df_modeltrain.drop(['Year', 'G', 'MP'], axis=1)
df_modeltest = df_modeltest.drop(['Year', 'G', 'MP'], axis=1)

##for understanding the final, cleaned data 
df_modeltrain.to_csv('train.csv')
df_modeltest.to_csv('test.csv')

##resetting the indexes
df_modeltrain = df_modeltrain.reset_index(drop=True)
df_modeltest = df_modeltest.reset_index(drop=True)

[0, 105, 213, 291, 380, 476, 559, 644, 724, 806]
[0, 22, 46, 81, 112, 144, 175, 204, 240, 290, 351, 411, 472, 540, 608, 675, 738, 801, 871, 943, 1026, 1098, 1160, 1234, 1309, 1381, 1459, 1527, 1606, 1686, 1772, 1869, 1965, 2056, 2146, 2244, 2341, 2449, 2557, 2654, 2721, 2813, 2891, 2983, 3075, 3179, 3282, 3377, 3482, 3580]
0.424


In [354]:
##implementing SGD classifier 
import sklearn 
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
X_train = df_modeltrain.drop(columns =['MVP'])
Y_train = df_modeltrain['MVP']
X_test = df_modeltest.drop(columns =['MVP'])
Y_test = df_modeltest['MVP']

##using logistic regression
model1 = SGDClassifier(loss = 'log_loss', penalty='l1')

##applying incremental learning onto each years dataset seperately 
for i in range(len(listofyeartrain)-1):
    X_batch = X_train[listofyeartrain[i]:listofyeartrain[i+1]]
    Y_batch = Y_train[listofyeartrain[i]:listofyeartrain[i+1]]

    model1.partial_fit(X_batch,Y_batch, classes = np.unique(Y_batch))


predictions = model1.predict(X_test)
predictions = list(predictions)
df_new = pd.DataFrame(predictions)

score = accuracy_score(Y_test, predictions)

##this accuracy is misrepresentative, as gains accuracy for not picking an mvp as well 
## since most of the players are not MVPs, this inflates the accuracy rate 
print(score)

listtostore = []
indexnum = 0

##finding out what is predicted
for i in range(len(predictions)):
    indexnum = indexnum+1
    if predictions[i] == 1:
        listtostore.append(indexnum)
        

for i in range(len(listtostore)):
    listtostore[i] = df_tracktest.iloc[listtostore[i], 13]

##finding real accuracy 
listofactualmvps = ['Lebron James', 'Derrick Rose', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo']


print(listofactualmvps)
print(listtostore)


0.9887387387387387
['Lebron James', 'Derrick Rose', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo']
[]


In [356]:
##creating a decision tree model
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
##use the MVP column as the Y 
X_train = df_modeltrain.drop(columns =['MVP'])
Y_train = df_modeltrain['MVP']
X_test = df_modeltest.drop(columns =['MVP'])
Y_test = df_modeltest['MVP']
model = DecisionTreeClassifier()

model.fit(X_train,Y_train)
predictions2 = model.predict(X_test)
score2 = accuracy_score(Y_test, predictions)
##serves as a misrepresentation ^^ mentioned above 
print(score2)

predictions2 = list(predictions2)


df_new2 = pd.DataFrame(predictions2)



listtostore2 = []
indexnum2 = 0
for i in range(len(predictions2)):
    indexnum2 = indexnum2+1
    if predictions2[i] == 1:
        listtostore2.append(indexnum2)

for i in range(len(listtostore2)):
    listtostore2[i] = df_tracktest.iloc[listtostore2[i], 13]

print(listofactualmvps)
print(listtostore2)


0.9887387387387387
['Lebron James', 'Derrick Rose', 'LeBron James', 'LeBron James', 'Kevin Durant', 'Stephen Curry', 'Stephen Curry', 'Russell Westbrook', 'James Harden', 'Giannis Antetokounmpo']
['Kwame Brown', 'Chris Duhon', 'Travis Outlaw', 'Ben Wallace', 'Earl Barron', 'Al Horford', 'Peja Stojaković', 'Tyrus Thomas', 'Devin Ebanks', 'Armon Johnson', 'Kemba Walker', 'Reggie Williams', 'Lou Amundson', 'Jae Crowder', 'Andre Drummond', 'Blake Griffin', 'Grant Hill', 'John Lucas III', 'Josh McRoberts', 'Tim Ohlbrecht', 'Dexter Pittman']
